$$
ELBO = \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{KL}(q(z|x) || p(z))
$$
- 要让 `q(z|x)` 接近 `p(z)` 以最小化 KL 散度，最简单的方法就是让 `q(z|x)` 与输入 `x` 无关。也就是说，无论输入什么 `x`，编码器都输出同一个标准正态分布。
- KL 散度为 0：当 q(z|x) = p(z) 时，D_KL(q(z|x) || p(z)) = 0。KL 损失降到了最低。
- 解码器“自力更生”：由于 z 不再包含任何关于 x 的信息（信息流被切断），解码器 p(x|z) 为了尽可能地降低重构误差，只能学习一个“万能”的解码策略。它会忽略 z，并尝试生成一个能够“平均”所有可能输入的 x 的输出。对于人脸数据集，这就是一张模糊的“平均脸”。

- 为什么会发生后验坍缩？
- 强大的解码器 (Powerful Decoder)：如果解码器能力过强（例如，使用了非常深或复杂的网络，如 Autoregressive 模型），它可能强大到不需要 z 的信息就能很好地“猜测”出 x 的样子。这给了模型忽略 z 的“资本”。
- KL 损失过强或引入过早：在训练初期，编码器和解码器都还很弱。如果此时 KL 损失的权重（β）过大，模型会发现降低 KL 损失比学习如何有效编解码要容易得多，于是就直接“躺平”，让 KL 损失降为 0。这就是 KL 退火 (KL Annealing) 发挥作用的地方。
- 数据本身的特性：如果数据非常简单或高度重复，模型也更容易学会“平均”策略。

如何判断是否发生了后验坍缩？
在训练过程中，你可以通过监控以下指标来判断：

- KL 散度值：最直接的指标。如果 L_KL 在训练开始后不久就迅速下降到非常接近 0 的值（例如 1e-5），并且在整个训练过程中都保持在这个水平，那么几乎可以肯定是发生了后验坍缩。
- 隐变量的利用率：
    - 从测试集中取一个样本 x，通过编码器得到其均值 μ 和方差 σ。
    - 在 μ 周围随机采样几个 z，送入解码器。
    - 如果生成的图像几乎没有变化，说明解码器忽略了 z 的变化。
    - 如果生成的图像有合理的、平滑的变化，说明模型健康。
- 生成样本的多样性：从先验分布 p(z) (即标准正态分布) 中随机采样 z，送入解码器生成新样本。如果生成的样本千篇一律，都是那张模糊的“平均脸”，这也是后验坍缩的典型症状。